# Detection of modified images or videos using Neural Networks

## Importing the libraries

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import io
import shutil
from kaggle.api.kaggle_api_extended import KaggleApi
import pywt

import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from keras.models import Sequential, Model, load_model
from keras.optimizers import Adam
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation, Input, GlobalAveragePooling2D
from keras.applications import ResNet50, MobileNet, MobileNetV2, VGG16, Xception, EfficientNetB1, EfficientNetB2, EfficientNetB3, EfficientNetV2B1
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

K.clear_session()
tf.compat.v1.reset_default_graph()
tf.compat.v1.enable_eager_execution()

## Downloading the dataset

In [ ]:
URL = "sophatvathana/casia-dataset"
PATH_DATASET = './../dataset/'

def download_dataset():
    api = KaggleApi()
    api.authenticate()
    print("Downloading files...")
    api.dataset_download_files('sophatvathana/casia-dataset', path=PATH_DATASET, unzip=True)

    print("\rDownload complete.")


def clean_directory():
    print("Moving folder...")
    os.rename(PATH_DATASET+"CASIA2/Au", PATH_DATASET+"Au")
    os.rename(PATH_DATASET+"CASIA2/Tp", PATH_DATASET+"Tp")
    
    print("Cleaning directory...")
    shutil.rmtree(PATH_DATASET+"casia")
    shutil.rmtree(PATH_DATASET+"CASIA1")
    shutil.rmtree(PATH_DATASET+"CASIA2")
    os.remove(PATH_DATASET+"Tp/Thumbs.db")
    os.remove(PATH_DATASET+"Au/Thumbs.db")
    print("Cleaning complete.")

def remove_images():
    print("Removing images...")
    for i in range(1, 4):
        os.remove(PATH_DATASET+"Au/Au ("+str(i)+").jpg")
        os.remove(PATH_DATASET+"Tp/Tp ("+str(i)+").jpg")
    print("Removing complete.")

In [ ]:
if not os.path.exists(PATH_DATASET+"Au"):
    download_dataset()
    clean_directory()
else:
    print("Dataset already Downloaded.")

In [ ]:
REAL_IMAGE_PATH = '../dataset/Au'
FAKE_IMAGE_PATH = "../dataset/Tp"
IMG_SIZE = (256, 256)
CLASS = ["real", "fake"]

## Loading the dataset

In [ ]:
cabezera = "category", "image", "class"
df_au = pd.DataFrame(columns=cabezera)
df_tp = pd.DataFrame(columns=cabezera)

for idx, file in enumerate(os.listdir(REAL_IMAGE_PATH)):
    img_path = os.path.join(REAL_IMAGE_PATH, file)
    category = file.split("_")

    df_au = pd.concat([df_au, pd.DataFrame([[category[1], img_path, CLASS[0]]], columns=cabezera)], ignore_index=True)

for file in os.listdir(FAKE_IMAGE_PATH):
    #convert image to np array
    img_path = os.path.join(FAKE_IMAGE_PATH, file)

    category = file.split("_")
    category[5] = category[5][:3]
    df_tp = pd.concat([df_tp, pd.DataFrame([[category[5], img_path, CLASS[1]]], columns=cabezera)], ignore_index=True)

df = pd.concat([df_au, df_tp], ignore_index=True)
df = df[df.category != "txt"]
df = df[df.category != "ind"]
df = df.sample(frac=1).reset_index(drop=True)

df.head()


## Creating Preprocessing Function

In [ ]:
def wavelet(image):
    # Convertir la imagen a escala de grises
    gris = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Aplicar la transformada de wavelet
    coeffs = pywt.dwt2(gris, 'haar')
    LL, (LH, HL, HH) = coeffs
    
    # Aplicar la transformada inversa de wavelet
    reconstruccion = pywt.idwt2(coeffs, 'haar')
    
    # Calcular la resta entre la imagen original y la reconstrucción
    resta = gris - reconstruccion
    
    # Convertir la resta a una imagen de 3 dimensiones
    resta_3d = np.stack((resta, resta, resta), axis=2).astype(np.uint8)
    
    return resta_3d



def preprocess_image(image):
    #image = tf.cast(tensor, tf.uint8).numpy()
    image = wavelet(image)
    #image = preprocess_input(image)
    return tf.convert_to_tensor(image)

## Creating Model

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
batch_size = 32

### Creating and splitting the dataset

In [ ]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_image,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_image,
)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=df_train,
    directory=None,
    x_col="image",
    y_col="class",
    subset="training",
    batch_size=batch_size,
    seed=42,
    shuffle=True,
    class_mode="binary",
    target_size=IMG_SIZE
)

val_generator = train_datagen.flow_from_dataframe(
    dataframe=df_train,
    directory=None,
    x_col="image",
    y_col="class",
    subset="validation",
    batch_size=batch_size,
    seed=42,
    shuffle=True,
    class_mode="binary",
    target_size=IMG_SIZE
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=df_test,
    directory=None,
    x_col="image",
    y_col="class",
    batch_size=batch_size,
    shuffle=False,
    class_mode="binary",
    target_size=IMG_SIZE
)

### Creating the checkpoints

In [ ]:
metrics = [
    tf.keras.metrics.BinaryAccuracy(name='binary_accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='auc'),
    tf.keras.metrics.AUC(name='prc', curve='PR')
]

early_stopping = EarlyStopping(
    monitor='val_loss', 
    min_delta=0, 
    patience=10, 
    verbose=0, 
    mode='auto', 
    baseline=None, 
    restore_best_weights=False
)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1)


model_chekpoint = ModelCheckpoint(
    filepath='./../model/checkpoints', 
    monitor='val_loss', 
    verbose=0, 
    save_best_only=True,
    save_weights_only=True, 
    mode='auto', 
    save_freq='epoch'
)

tensor_board = TensorBoard(
    log_dir='./../model/logs',
    histogram_freq=0,
    write_graph=True,
    write_images=False,
    update_freq='epoch',
    profile_batch=2,
    embeddings_freq=0,
    embeddings_metadata=None
)

callbacks = [early_stopping, model_chekpoint, tensor_board]
optimizer = Adam(1e-3)
loss = keras.losses.BinaryCrossentropy()

### Creating the model fron scratch

detect_manipulated_images_model_scratch_v1.h5 10min 14s 15 epocas bacth_size=32 42s

loss: 0.6755 - binary_accuracy: 0.5969 - precision: 0.5969 - recall: 1.0000 - auc: 0.5000 - prc: 0.5969

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), activation=None, input_shape=(256, 256, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(5, 5), activation=None, kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(5, 5), activation=None, kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(256, activation=None, kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

### Creating the model using transfer learning (ResNet50)

detect_manipulated_images_model_resNet50.h5 21m 14s 14 epocas bacth_size=32 90s

loss: 0.6753 - binary_accuracy: 0.5969 - precision: 0.5969 - recall: 1.0000 - auc: 0.5000 - prc: 0.5969

In [ ]:
inputs = Input(shape=(256, 256, 3))

model = ResNet50(weights='imagenet', include_top=False, input_tensor=inputs)

x = Flatten()(model.output)
x = Dense(256, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=model.inputs, outputs=predictions)

### Creating the model using transfer learning (MobileNet)

detect_manipulated_images_model_mobilenet.h5 22m 5s 20 epocas bacth_size=32 67s

loss: 0.6755 - binary_accuracy: 0.5969 - precision: 0.5969 - recall: 1.0000 - auc: 0.5000 - prc: 0.5969

In [ ]:
inputs = Input(shape=(256, 256, 3))
model = MobileNet(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

x = GlobalAveragePooling2D()(model.output)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=model.inputs, outputs=predictions)

### Creating the model using transfer learning (VGG16)

detect_manipulated_images_model_vgg16_v1.h5 36m 15s 20 epocas bacth_size=32 108s

loss: 0.6651 - binary_accuracy: 0.6180 - precision: 0.6180 - recall: 1.0000 - auc: 0.5000 - prc: 0.6180

In [ ]:
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

x = vgg16.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=vgg16.inputs, outputs=predictions)

### Creatin Model Xception

detect_manipulated_images_model_Xception.h5 29m 5s 11 epocas bacth_size=32 158s


loss: 0.6055 - binary_accuracy: 0.6180 - precision: 0.6180 - recall: 1.0000 - auc: 0.5000 - prc: 0.6180

In [ ]:
inputs = Input(shape=(256, 256, 3))
model = Xception(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

x = GlobalAveragePooling2D()(model.output)
x = Dense(256, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=model.inputs, outputs=predictions)

### Creatin Model EfficientNetB1

detect_manipulated_images_model_EfficientNetB1.h5 31m 38s 14 epocas bacth_size=32 125s

loss: 0.6684 - binary_accuracy: 0.6135 - precision: 0.6142 - recall: 0.9973 - auc: 0.4908 - prc: 0.6027


In [ ]:
model = EfficientNetB1(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

x = GlobalAveragePooling2D()(model.output)
x = Dense(256, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=model.inputs, outputs=predictions)

### Training the model

In [ ]:
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
history = model.fit(train_generator, epochs=20, batch_size=batch_size, validation_data=val_generator, callbacks=callbacks)

### Evaluating the model

In [ ]:
model.evaluate(test_generator)

### Showing the results

In [ ]:
def plot_metrics(history):
    metrics = ['binary_accuracy', 'loss', 'prc', 'precision', 'recall']
    fig, axes = plt.subplots(len(metrics), 1, figsize=(10, 10))
    
    for i, metric in enumerate(metrics):
        axes[i].plot(history.history[metric], label='train')
        axes[i].plot(history.history[f'val_{metric}'], label='val')
        axes[i].set_title(metric)
        axes[i].legend()
    
    plt.tight_layout()
    plt.show()

def plot_confusion_matrix(model, X, y_true):
    y_pred = model.predict(X) > 0.5
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6,6))
    plt.imshow(cm, cmap=plt.cm.Reds)
    plt.title('Confusion Matrix', fontsize=16)
    plt.ylabel('True label', fontsize=14)
    plt.xlabel('Predicted label', fontsize=14)
    plt.xticks([0, 1], ['Manipulated', 'Original'], fontsize=12)
    plt.yticks([0, 1], ['Manipulated', 'Original'], fontsize=12)
    plt.colorbar()
    for i in range(2):
        for j in range(2):
            plt.text(j, i, str(cm[i][j]), ha='center', va='center', fontsize=20)
    plt.show()

In [ ]:
plot_metrics(history)

In [ ]:
model.save('./../model/wavelet_models/detect_manipulated_images_model_efficientNetB1.h5')